In [1]:
from absl import app, flags
from easydict import EasyDict
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
from torchvision import models
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, models, transforms

from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.torch.attacks.carlini_wagner_l2 import carlini_wagner_l2
from cleverhans.torch.attacks.hop_skip_jump_attack import hop_skip_jump_attack
from cleverhans.torch.attacks.noise import noise
# from cleverhans.torch.attacks.semantic import semantic
from cleverhans.torch.attacks.sparse_l1_descent import sparse_l1_descent
from cleverhans.torch.attacks.spsa import spsa



In [2]:
imagenet_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

device = "cuda" if torch.cuda.is_available() else "cpu"

function_labels = ['real', 'fgm', 'pgd', 'noise', 'semantic', 'l1', 'spsa', 'l2', 'hsp']
categories = ["cats", "birds", "fruits", "veggies", "vehicles"]
                #["dogs", "cats", "birds", "fruits", "veggies", "vehicles"]
models_names = ["alexnet", "vgg", "googlenet", "squeezenet", "densenet", "inception"]

images_path = r".\data\imagenet-subset/"
models_save_path = r".\adversarial-data\imagenet/" # {categories}/{model_name}  

imgnet_labels_path = r"./data/imagenet/imagenet-labels.txt"
with open(imgnet_labels_path) as f:
    data = f.read()   
imagenet_labels = json.loads(data)

class_labels = EasyDict(basset=161, beagle=162, border_collie=232, 
                        english_foxhound=167, malamute=249, maltese_dog=153, 
                        redbone=168, egyptian=285, persian=283, siamese=284,
                        tabby=281, tiger=282, albatross=146, bald_eagle=22,
                        flamingo=130, goldfinch=11, great_grey_owl=24, 
                        house_finch=12, magpie=18, ostrich=9, peacock=84,
                        toucan=96, banana=954, fig=952, lemon=951, 
                        pineapple=953, pomegranate=957, strawberry=949,
                        bell_pepper=945, cabbage=936, cauliflower=938,
                        courgette=939, cucumber=943, limousine=627, 
                        motorboat=814, snowmobile=802, tank=847, taxicab=468)

In [3]:
# ----- MODELS ------
all_models = []

all_models.append(models.alexnet(pretrained=True))
all_models.append(models.vgg11(pretrained=True))
all_models.append(models.googlenet(pretrained=True))
all_models.append(models.squeezenet1_0(pretrained=True))
all_models.append(models.densenet161(pretrained=True))
all_models.append(models.inception_v3(pretrained=True))

In [4]:
def convert_img(np_img, is_tensor=True):
    if is_tensor:
        np_img = np_img[0].permute(1,2,0).cpu().detach().numpy() # 4d image expected [batch_size, colour, w, h]
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    out_img = std * np_img + mean
    out_img = np.clip(out_img, 0, 1)
    return out_img

def plot_time_taken(report, labels, path):
    steps = len(report.time) // report.funcs_tested  # should always be an integer
    
    x = range(steps)
    y = []
    
    for i in range(report.funcs_tested):
        time_taken = report.time[i]
        current_y = [time_taken] # 1 image.

        for j in range(1, steps):
            current_y.append(current_y[j-1] + time_taken) # 2 images onwards.
        y.append(current_y)
        
    
    plt.figure(figsize=(10, 5))
    for i, label in enumerate(labels):
        plt.plot(x, y[i], label=label)
    plt.xlabel("Number of images")
    plt.ylabel("Time taken (seconds)")
    plt.legend()
    plt.savefig("{0}/time-taken.png".format(path))

    
def convert_to_imagenet_idx(y_trues, class_names, class_labels):
    adjusted_y_true = []
    for y_true in y_trues:
        adjusted_y_true.append(class_labels[class_names[y_true]])
        
    return torch.Tensor(adjusted_y_true)
    

In [5]:
#plot_time_taken(alexnet_report, labels=function_labels[:alexnet_report.funcs_tested], path=alexnet_dog_path)

In [6]:
def base(net, x, y, report):
    _, y_pred = net(x).max(1)  # model prediction on clean examples
    report.y_pred.append(y_pred)
    report.correct += y_pred.eq(y).sum().item()
    
    return x, y_pred

def fgm(net, x, y, report, eps=0.3):
    start = time.time()
    x_fgm = fast_gradient_method(net, x, eps, np.inf)
    _, y_pred_fgm = net(x_fgm).max(1)  # model prediction on FGM adversarial examples
    end = time.time()
    
    #print("FGM complete")
    
    report.time.append(end-start)
    report.y_pred_fgm.append(y_pred_fgm)
    report.correct_fgm += y_pred_fgm.eq(y).sum().item()
    
    return x_fgm, y_pred_fgm

def pgd(net, x, y, report, eps=0.3):
    start = time.time()
    x_pgd = projected_gradient_descent(net, x, eps, 0.01, 40, np.inf)
    _, y_pred_pgd = net(x_pgd).max(1)  # model prediction on PGD adversarial examples
    end = time.time()
 
    #print("PGD complete")
    report.time.append(end-start)
    report.y_pred_pgd.append(y_pred_pgd)
    report.correct_pgd += y_pred_pgd.eq(y).sum().item()
    
    return x_pgd, y_pred_pgd

def own_noise(net, x, y, report):
    start = time.time()
    x_noise = noise(x)
    _, y_pred_noise = net(x_noise).max(1)
    end = time.time()
    
    #print("NOISE complete")
    report.time.append(end-start)
    report.y_pred_noise.append(y_pred_noise)
    report.correct_noise += y_pred_noise.eq(y).sum().item()

    return x_noise, y_pred_noise


def semantic(net, x, y, report):
    def sem(x, center=True, max_val=1.0):
        if center:
            return x * -1
        return max_val - x
    
    start = time.time()
    x_semantic = sem(x, center=False)  # latest pip package has outdated function.
    _, y_pred_semantic = net(x_semantic).max(1)
    end = time.time()
    
    #print("SEMANTIC complete")
    report.time.append(end-start)
    report.y_pred_semantic.append(y_pred_semantic)
    report.correct_semantic += y_pred_semantic.eq(y).sum().item()

    return x_semantic, y_pred_semantic

def l1(net, x, y, report):
    start = time.time()
    x_l1 = sparse_l1_descent(net, x)
    _, y_pred_l1 = net(x_l1).max(1)
    end = time.time()
    
    #print("L1 complete")
    report.time.append(end-start)
    report.y_pred_l1.append(y_pred_l1)
    report.correct_l1 += y_pred_l1.eq(y).sum().item()    


    
    return x_l1, y_pred_l1


def own_spsa(net, x, y, report, eps=0.3):
    start = time.time()
    x_spsa = spsa(net, x, eps, nb_iter=3, sanity_checks=False)
    _, y_pred_spsa = net(x_spsa).max(1)
    end = time.time()
    
    #print("SPSA complete")
    report.time.append(end-start)
    report.y_pred_spsa.append(y_pred_spsa)
    report.correct_spsa += y_pred_spsa.eq(y).sum().item() 

    
    return x_spsa, y_pred_spsa
   


def l2(net, x, y, report):
    start = time.time()
    x_l2 = carlini_wagner_l2(net, x, n_classes=1000)
    _, y_pred_l2 = net(x_l2).max(1)
    end = time.time()
    
    #print("L2 complete")
    report.time.append(end-start)
    report.y_pred_l2.append(y_pred_l2)
    report.correct_l2 += y_pred_l2.eq(y).sum().item()

    return x_l2, y_pred_l2


def hsp(net, x, y, report):
    start = time.time()
    x_hop_skip = hop_skip_jump_attack(net, x, np.inf, verbose=False)
    _, y_pred_hop_skip = net(x_hop_skip).max(1)
    end = time.time()
    #print("HSP complete")
    
    report.time.append(end-start)
    report.y_pred_hop_skip.append(y_pred_hop_skip)
    report.correct_hop_skip += y_pred_hop_skip.eq(y).sum().item()
    
    return x_hop_skip, y_pred_hop_skip
   
        
    

In [7]:
def test_model(net, dataloader, path, class_names, class_labels, save=False):
    r = EasyDict(nb_test=0, correct=0, correct_fgm=0, correct_pgd=0, 
                          correct_l2=0, correct_hop_skip=0, correct_noise=0,
                         correct_semantic=0, correct_l1=0, correct_spsa=0,
                         y_pred=[], y_pred_fgm=[], y_pred_pgd=[],
                         y_pred_l2=[], y_pred_hop_skip=[], y_pred_noise=[], y_pred_semantic=[],
                         y_pred_l1=[], y_pred_spsa=[], time=[], funcs_tested=0)

    for j, (x, y) in enumerate(tqdm(dataloaders)):
        adversarial_outputs = []
        x, y = x.to(device), y.to(device)
        r.nb_test += y.size(0)
        
        # convert y to image net labels.
        adjusted_y = convert_to_imagenet_idx(y, class_names, class_labels)
        adjusted_y = adjusted_y.to(device)
        
        adversarial_outputs.append(base(net, x, adjusted_y, r))
        adversarial_outputs.append(fgm(net, x, adjusted_y, r))
        adversarial_outputs.append(pgd(net, x, adjusted_y, r))
        adversarial_outputs.append(own_noise(net, x, adjusted_y, r))
        adversarial_outputs.append(semantic(net, x, adjusted_y, r))
        adversarial_outputs.append(l1(net, x, adjusted_y, r))
        #adversarial_outputs.append(own_spsa(net, x, adjusted_y, r))
        
        #adversarial_outputs.append(l2(net, x, adjusted_y, r))
        #adversarial_outputs.append(hsp(net, x, adjusted_y, r))

        if save:
            fig, axs = plt.subplots(1, len(adversarial_outputs),figsize=(15,15))
            
            for i, (x_adv_img, y_adv_pred) in enumerate(adversarial_outputs):  # assumes BATCHSIZE=1.
                out_img = convert_img(x_adv_img)
                axs[i].xaxis.set_ticks([])
                axs[i].yaxis.set_ticks([])
                axs[i].set_xlabel(imagenet_labels[y_adv_pred][:10])
                axs[i].set_title(function_labels[i])  # clip to 10 characters
                axs[i].imshow(out_img)     
                
            plt.savefig("{0}/{1}-{2}.png".format(path, class_names[y], j))

#         print("test acc on clean examples (%): {:.3f}".format(r.correct / r.nb_test * 100.0))
#         print("test acc on FGM adversarial examples (%): {:.3f}".format(r.correct_fgm / r.nb_test * 100.0))
#         print("test acc on PGD adversarial examples (%): {:.3f}".format(r.correct_pgd / r.nb_test * 100.0))
#         print("test acc on L2 adversarial examples (%): {:.3f}".format(r.correct_l2 / r.nb_test * 100.0))
#         print("test acc on HOP-SKIP-JUMP adversarial examples (%): {:.3f}".format(r.correct_hop_skip / r.nb_test * 100.0))
#         print("test acc on NOISE adversarial examples (%): {:.3f}".format(r.correct_noise / r.nb_test * 100.0))
#         print("test acc on SEMANTIC adversarial examples (%): {:.3f}".format(r.correct_semantic / r.nb_test * 100.0))
#         print("test acc on L1 adversarial examples (%): {:.3f}".format(r.correct_l1 / r.nb_test * 100.0))
#         print("test acc on SPSA adversarial examples (%): {:.3f}".format(r.correct_spsa / r.nb_test * 100.0))

    with open("{0}/report.txt".format(path), 'w+') as f:
        for key in r:
            f.write("{0}: {1}\n".format(key, r[key]))
        f.write("test acc on clean examples (%): {:.2f}\n".format(r.correct / r.nb_test * 100.0))
        f.write("test acc on FGM adversarial examples (%): {:.2f}\n".format(r.correct_fgm / r.nb_test * 100.0))
        f.write("test acc on PGD adversarial examples (%): {:.2f}\n".format(r.correct_pgd / r.nb_test * 100.0))
        f.write("test acc on L2 adversarial examples (%): {:.2f}\n".format(r.correct_l2 / r.nb_test * 100.0))
        f.write("test acc on HOP-SKIP-JUMP adversarial examples (%): {:.2f}\n".format(r.correct_hop_skip / r.nb_test * 100.0))
        f.write("test acc on NOISE adversarial examples (%): {:.2f}\n".format(r.correct_noise / r.nb_test * 100.0))
        f.write("test acc on SEMANTIC adversarial examples (%): {:.2f}\n".format(r.correct_semantic / r.nb_test * 100.0))
        f.write("test acc on L1 adversarial examples (%): {:.2f}\n".format(r.correct_l1 / r.nb_test * 100.0))
        f.write("test acc on SPSA adversarial examples (%): {:.2f}\n".format(r.correct_spsa / r.nb_test * 100.0))

    print("test acc on clean examples (%): {:.3f}".format(r.correct / r.nb_test * 100.0))
    print("test acc on FGM adversarial examples (%): {:.3f}".format(r.correct_fgm / r.nb_test * 100.0))
    print("test acc on PGD adversarial examples (%): {:.3f}".format(r.correct_pgd / r.nb_test * 100.0))
    print("test acc on L2 adversarial examples (%): {:.3f}".format(r.correct_l2 / r.nb_test * 100.0))
    print("test acc on HOP-SKIP-JUMP adversarial examples (%): {:.3f}".format(r.correct_hop_skip / r.nb_test * 100.0))
    print("test acc on NOISE adversarial examples (%): {:.3f}".format(r.correct_noise / r.nb_test * 100.0))
    print("test acc on SEMANTIC adversarial examples (%): {:.3f}".format(r.correct_semantic / r.nb_test * 100.0))
    print("test acc on L1 adversarial examples (%): {:.3f}".format(r.correct_l1 / r.nb_test * 100.0))
    print("test acc on SPSA adversarial examples (%): {:.3f}".format(r.correct_spsa / r.nb_test * 100.0))
    r.funcs_tested = len(adversarial_outputs)
    
    
    return r

In [ ]:
for category in tqdm(categories):
    current_category_folder = r"{0}/{1}".format(images_path, category)
    image_datasets = datasets.ImageFolder(os.path.join(current_category_folder), imagenet_transform)
    dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=1, shuffle=True, num_workers=4)
    current_class_names = image_datasets.classes
    
    reports = []
    for i, model in enumerate(all_models):
        print("Category: {0} | Model: {1}".format(category, models_names[i]))
        model.to(device)
        model.eval()
        
        current_model_save_path = "{0}/{1}/{2}/".format(models_save_path, category, models_names[i])
        os.makedirs(os.path.dirname(current_model_save_path), exist_ok=True)
        
        current_report = test_model(model, dataloaders, current_model_save_path, 
                                    current_class_names, class_labels, save=True)
        reports.append(current_report)
        
        
        
    

Category: cats | Model: alexnet


In [7]:
from torchsummary import summary

In [9]:
for m in all_models:
    summary(m.cuda(), (3, 227, 227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

           Conv2d-96          [-1, 128, 14, 14]          65,536
      BatchNorm2d-97          [-1, 128, 14, 14]             256
      BasicConv2d-98          [-1, 128, 14, 14]               0
           Conv2d-99          [-1, 256, 14, 14]         294,912
     BatchNorm2d-100          [-1, 256, 14, 14]             512
     BasicConv2d-101          [-1, 256, 14, 14]               0
          Conv2d-102           [-1, 24, 14, 14]          12,288
     BatchNorm2d-103           [-1, 24, 14, 14]              48
     BasicConv2d-104           [-1, 24, 14, 14]               0
          Conv2d-105           [-1, 64, 14, 14]          13,824
     BatchNorm2d-106           [-1, 64, 14, 14]             128
     BasicConv2d-107           [-1, 64, 14, 14]               0
       MaxPool2d-108          [-1, 512, 14, 14]               0
          Conv2d-109           [-1, 64, 14, 14]          32,768
     BatchNorm2d-110           [-1, 64, 14, 14]             128
     BasicConv2d-111           [-1, 64, 

AttributeError: 'list' object has no attribute 'size'

In [17]:
len(all_models)

6

In [23]:
print(all_models[0])

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
summary(all_models[6].cuda(), (3, 227, 227))

IndexError: list index out of range